Вам предоставлен набор данных, первоначально полученный в Национальном институте диабета, болезней органов пищеварения и почек. 

Этот набор данных создан для того, чтобы на основе определённых диагностических измерений предсказать, есть ли у пациента диабет. 

На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. 
В частности, все пациенты здесь — женщины не моложе 21 года индейского происхождения Пима.

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

In [32]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


Pregnancies — количество беременностей.

Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.

BloodPressure — диастолическое артериальное давление (мм рт. ст.).

SkinThickness — толщина кожной складки трицепса (мм).

Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).

BMI — индекс массы тела (\(\frac{вес\ в\ кг}{(рост\ в\ м)^2}\)).

DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).

Age — возраст.

Outcome — наличие диабета (0 — нет, 1 — да).


In [33]:
display(diabetes.describe())

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000
mean,3.848329,120.822622,69.035990,20.457584,79.521851,31.982262,0.470871,33.317481,0.344473
std,3.360782,31.883264,19.432323,15.954452,114.862405,7.853917,0.330669,11.816300,0.475502
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.241500,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,27.000000,32.000000,0.370000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,126.750000,36.500000,0.625500,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [34]:
display(diabetes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


None

In [35]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [36]:
# удалим признак
dia_data = diabetes.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {dia_data.shape[1]}')

Результирующее число признаков: 9


In [37]:
# 8.1 найти и удалить дубликаты
# найдем дубликаты убрав id
dupl_columns = list(dia_data.columns)
#dupl_columns.remove('id')

mask = dia_data.duplicated(subset=dupl_columns)
dia_duplicates = dia_data[mask]
print(f'Число найденных дубликатов: {dia_duplicates.shape[0]}')

Число найденных дубликатов: 10


In [38]:
dia_data = dia_data.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {dia_data.shape[0]}')

Результирующее число записей: 768


In [39]:
# ищем пропуски
display(dia_data.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [42]:
# заменяем 0 на nan 
lst_col = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
for col in lst_col:
    dia_data[col]=dia_data[col].apply(lambda x: np.nan if (x==0) else x)

# ищем пропуски
display(dia_data.isnull().mean())

Pregnancies                 0.000000
Glucose                     0.006510
BloodPressure               0.045573
SkinThickness               0.295573
Insulin                     0.486979
BMI                         0.014323
DiabetesPedigreeFunction    0.000000
Age                         0.000000
Outcome                     0.000000
dtype: float64

In [44]:
# Удалите из данных признаки, где число пропусков составляет более 30 %. 
# Сколько признаков осталось в ваших данных 

lst_col = list(dia_data.columns)
del_col = []
for col in lst_col:
    if dia_data[col].isnull().mean() > 0.3:
        del_col.append(col)
dia_data = dia_data.drop(del_col, axis=1)
print(f'Результирующее число признаков: {dia_data.shape[1]}') 

Результирующее число признаков: 8


In [45]:
#Удалите из данных только те строки, 
# в которых содержится более двух пропусков одновременно. 
# Чему равно результирующее число записей в таблице?

#отбрасываем строки с числом пропусков более 2 в строке
n = dia_data.shape[1] #число столбцов
dia_data = dia_data.dropna(thresh=n-2, axis=0)
print(f'Результирующее число строк: {dia_data.shape[0]}') 

Результирующее число строк: 761


In [48]:
dia_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 761 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               761 non-null    int64  
 1   Glucose                   756 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   BMI                       757 non-null    float64
 5   DiabetesPedigreeFunction  761 non-null    float64
 6   Age                       761 non-null    int64  
 7   Outcome                   761 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 53.5 KB


In [50]:
# В оставшихся записях замените пропуски на медиану. 
# Чему равно среднее значение в столбце SkinThickness?

#создаём словарь 'имя_столбца': число (признак), на который надо заменить пропуски 
values = {
    'Pregnancies': dia_data['Pregnancies'].median(),
    'Glucose': dia_data['Glucose'].median(),
    'BloodPressure': dia_data['BloodPressure'].median(),
    'SkinThickness': dia_data['SkinThickness'].median(),
    'BMI': dia_data['BMI'].median(),
    'DiabetesPedigreeFunction': dia_data['DiabetesPedigreeFunction'].median(),
    'Age': dia_data['Age'].median(),
    'Outcome': dia_data['Outcome'].median()
}

#заполняем оставшиеся записи константами в соответствии со словарем values
dia_data = dia_data.fillna(values)
dia_data['SkinThickness'].mean()

29.109067017082786

In [52]:
# Сколько выбросов найдёт классический метод 
# межквартильного размаха в признаке SkinThickness?

def outliers_iqr_mod(data, feature,left=1.5, right=1.5):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(dia_data, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')


Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [55]:
# Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?
def outliers_z_score_mod(data, feature, 
                         log_scale=False,
                         left=3,
                         right=3):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score_mod(dia_data, 
                                         'SkinThickness', 
                                         log_scale=False,
                                         left=3,
                                         right=3)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')



Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


In [59]:
def outliers_iqr_mod_log(data, 
                         feature,
                         log_scale=False,
                         left=1.5, 
                         right=1.5):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

In [60]:
# На приведённой гистограмме показано распределение 
# признака DiabetesPedigreeFunction. 
# Такой вид распределения очень похож на логнормальный, 
# и он заставляет задуматься о логарифмировании признака. 
# Найдите сначала число выбросов в признаке DiabetesPedigreeFunction 
# с помощью классического метода межквартильного размаха.

outliers, cleaned = outliers_iqr_mod_log(dia_data, 
                                         'DiabetesPedigreeFunction', 
                                         log_scale=False,
                                         left=1.5,
                                         right=1.5)
print(f'исло выбросов по методу Тьюки log=False: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

# Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). 
# Какова разница между двумя этими числами (вычтите из первого второе)?
outliers, cleaned = outliers_iqr_mod_log(dia_data, 
                                         'DiabetesPedigreeFunction', 
                                         log_scale=True,
                                         left=1.5,
                                         right=1.5)
print(f'исло выбросов по методу Тьюки log=True: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')



исло выбросов по методу Тьюки log=False: 29
Результирующее число записей: 732
исло выбросов по методу Тьюки log=True: 0
Результирующее число записей: 761
